In [3]:
#importing Libraries
import numpy as np
import pandas as pd
import pprint

eps = np.finfo(float).eps
data=pd.read_csv("lab01_dataset_1.csv")

def splitPoints(data):
    attributeDataFrame = pd.DataFrame({ 'Score' : data['Score'], 'Output' : data['Output']})
    attributeDataFrame.reindex(np.arange(len(attributeDataFrame['Score'].index)))
    print(attributeDataFrame)

    outputvalue = attributeDataFrame['Output'][0]

    splitPointDict = {}

    for i in range(1, len(attributeDataFrame['Score'])):
        if(attributeDataFrame['Output'][i] != outputvalue):
            # (i-1) (i) mean
            mean = (attributeDataFrame['Score'][i-1] + attributeDataFrame['Score'][i])/2
            splitPointDict[i-1] = mean
            outputvalue = attributeDataFrame['Output'][i]
        
    print(splitPointDict)

    return attributeDataFrame, splitPointDict

attributeDataFrame, splitPoint = splitPoints(data)

columnsList = {}

for i in splitPoint:
    split = splitPoint[i]
    # "Score_" + str(split)
    scoreList=[]
    for j in range(0, len(attributeDataFrame['Score'])):
        if attributeDataFrame['Score'][j] < split:
            scoreList.append(1)
        else: 
            scoreList.append(0)

    columnsList["Score_" + str(split)] = scoreList

columnsList



    Score Output
0      35    Yes
1      91     No
2      52     No
3      83     No
4      48     No
5      61     No
6      40    Yes
7      98     No
8      73    Yes
9      44    Yes
10     86     No
11     39    Yes
12     66     No
13     75    Yes
14     50     No
15     69     No
16     70    Yes
17     95     No
18     80    Yes
{0: 63.0, 5: 50.5, 6: 69.0, 7: 85.5, 9: 65.0, 10: 62.5, 11: 52.5, 12: 70.5, 13: 62.5, 15: 69.5, 16: 82.5, 17: 87.5}


{'Score_63.0': [1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
 'Score_50.5': [1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
 'Score_69.0': [1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0],
 'Score_85.5': [1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
 'Score_65.0': [1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
 'Score_62.5': [1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
 'Score_52.5': [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
 'Score_70.5': [1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0],
 'Score_69.5': [1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0],
 'Score_82.5': [1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1],
 'Score_87.5': [1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]}

In [4]:

# Entropy of the target attribute values
def find_entropy(df):
    target = df.keys()[-1]  # The last dataframe column is the target attribute (playGolf)
    entropy = 0
    values = df[target].unique()
    # for each value in the target playGolf attribute values
    for value in values:
        # ratio of values occurring and entropy
        fraction = df[target].value_counts()[value] / len(df[target])
        entropy += -fraction * np.log2(fraction)
    return entropy

# Entropy of attribute values
def find_entropy_attribute(df, attribute):
    target = df.keys()[-1]
    target_variables = df[target].unique()  
    # unique values in target playGolf attribute (Yes, No)
    variables = df[attribute].unique() # Identify Sunny, Overcast, Rainy
    # attribute entropy
    # Variables=[sunny, sunny....5, overcast1.....overcast4, Rainy1...Ra5 ]
    entropy2 = 0
    # for each attribute value in attribute values
    for variable in variables:
        
        # value entropy
        entropy = 0
        # for each target value in target values (yes/no)
        for target_variable in target_variables:
            
   # frequency of attribute and target values (boolean indexing, pandas dataframe filtering)
          num = len(df[attribute][df[attribute] == variable][df[target] == target_variable])
          den = len(df[attribute][df[attribute] == variable])
          fraction = num / (den + eps)
          entropy += -fraction * np.log2(fraction + eps)
        fraction2 = den / len(df)
        entropy2 += -fraction2 * entropy
    return abs(entropy2)


def bestClassifier(df):
    # Entropy_att = []
    # information gain array for all attributes
    IG = []
    # for all attributes excluding target
    for key in df.keys()[:-1]:
        # Entropy_att.append(find_entropy_attribute(df,key))
        # calculate and record information gain value
        IG.append(find_entropy(df) - find_entropy_attribute(df, key)) 
    return df.keys()[:-1][np.argmax(IG)]  



# function for creating subtable
def get_subtable(df, node, value):
    return df[df[node] == value].reset_index(drop=True)
def ID3split(df, tree=None):
    target = df.keys()[-1]
    
    # Here we build our decision tree
    # Get attribute with maximum information gain
    node = bestClassifier(df) # 0.247
    # Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attributeValues = np.unique(df[node])
    # Create an empty dictionary to create tree (recursive-friendly definition)
    if tree is None:               #  Outlook ->root node attribute
        tree = {}
        tree[node] = {}
    # following loop recursively calls ID3split to create and add to the tree
    # it runs till the tree is pure (leaf (result) node branches are added to the tree)
    for value in attributeValues:
        
        # get the subtable from current node based on the value
        subtable = get_subtable(df, node, value)
        # get the most common target value in the subtable
        targetValues, counts = np.unique(subtable[target], return_counts=True)
        
        # if the subtable is empty, assign the leaf node to the most common target value
        if len(counts) == 1:
            tree[node][value] = targetValues[0]
        
        else:
            # recursively call ID3 to create subtrees
            tree[node][value] = ID3split(subtable)  # Calling the function recursively
    return tree

decisionTree = ID3split(data)
pprint.pprint(decisionTree)

{'Score': {35: 'Yes',
           39: 'Yes',
           40: 'Yes',
           44: 'Yes',
           48: 'No',
           50: 'No',
           52: 'No',
           61: 'No',
           66: 'No',
           69: 'No',
           70: 'Yes',
           73: 'Yes',
           75: 'Yes',
           80: 'Yes',
           83: 'No',
           86: 'No',
           91: 'No',
           95: 'No',
           98: 'No'}}
